# Welcome To My Data Science Project on Neighborhood in Toronto

## Segmenting/Clustering  NeighborhoodS in toronto 
### Coursera Capstone Project 

In [26]:
!pip install lxml   

In [27]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import numpy as np
import json 


import lxml.html as lh
import bs4 as bs
import urllib.request


print('Libraries imported.')

Libraries imported.


In [14]:
#Save the URL 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### USING BEAUTIFULSOUP4 FOR SCRAPING  AND XLML FOR PARSING 

In [19]:
#using BeautifulSoup4 to scrap the Data 

def Table_Scraping_BS4 (cname, cols):
    url_page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(url_page, 'lxml')
    url_table = soup.find("table", class_ = cname)
    header = [head.findAll(text = True)[0].strip() for head in url_table.find_all("th")]
    data   = [[td.findAll(text = True)[0].strip() for td in tr.find_all("td")]
    for tr in  url_table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    _df = pd.DataFrame(data, columns = header)
    return _df

In [20]:
#parsing using xpath 

def Table_Scraping_lxml(XPATH, cols):
    url_page = requests.get(url)
    doc = lh.fromstring(url_page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
        for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        _df = pd.DataFrame(data, columns = headers)
        return _df


In [22]:
_data = Table_Scraping_BS4("wikitable", 3)

Cleaning Up The Scrapped Data

In [23]:
#ignores cells with unassigned Borough 
df = _data[~_data['Borough'].isin(['Not assigned'])]
#Sort and Reset Index 
df = df.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending = [1,1,1]).reset_index(drop = True)
#
df.loc[df['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = df['Borough']
check_unassigned_sample = df.loc[df['Borough'] == 'Queen\'s Park']
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [24]:
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [25]:
df.shape

(103, 3)